In [ ]:
"""
Goal: scrape kingfisher.kz website
Written by: Yedige Ashmet
Language: Python
"""

import re
import requests
import collections
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
#Getting Url request
url = "https://kingfisher.kz/"
page = requests.get(url)
page

<Response [200]>

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')
menu = []
for ultag in soup.find_all('ul', {'class': 'topMenu'}):
  for litag in ultag.find_all('a'):
    menu.append(litag.get('href'))

#Get all the links from the website and do not include actions 
soup_vals = []
for urls in menu[:-1]:
  soup_vals.append(url + urls[1:])

In [ ]:
#Iterate over urls and soup it
def get_country(string):
  value = ""
  for k in reversed(string.split("\n")):
    value += re.sub(r"[^a-zA-Z0-9]+", ' ', k)
    break
  return value


all_links = []
for urls in soup_vals:
  soup_1 = BeautifulSoup(requests.get(urls).content, 'html.parser')
  for a in soup_1.find_all('a',{'class':'title'}, href=True): 
    if a.text:
      all_links.append(url+a['href'][1:])

print(all_links)   

['https://kingfisher.kz/moreprodukty/krevetki/krevetka-korolevskaya-2630-ochischennaya/', 'https://kingfisher.kz/moreprodukty/krevetki/krevetka-korolevskaya-ochischennaya-31-40/', 'https://kingfisher.kz/moreprodukty/krevetki/langustiny-v-pancire-s-golovoy-razmery-21-30/', 'https://kingfisher.kz/moreprodukty/krevetki/krevetka-korolevskaya-2125-v-pancire-bez-golovy/', 'https://kingfisher.kz/bakaleya/aziatskaya-kuhnya/vodorosli/chuka-salat-zamorozhenniy/', 'https://kingfisher.kz/moreprodukty/krevetki/krevetka-arkticheskaya-7090-grenlandiya/', 'https://kingfisher.kz/moreprodukty/krevetki/krevetka-tigrovaya-1620-v-pancire/', 'https://kingfisher.kz/moreprodukty/krevetki/langustiny-v-pancire-s-golovoy-razmer-10-20/', 'https://kingfisher.kz/moreprodukty/krevetki/krevetka-salatnaya-100-200/', 'https://kingfisher.kz/moreprodukty/krevetki/krevetka-severnaya-90-120-grenlandiya/', 'https://kingfisher.kz/moreprodukty/krevetki/langustiny-v-pancire-bez-golovy-razmer-30-55/', 'https://kingfisher.kz/mor

In [ ]:
def country(string):
  vals = ' '.join(re.sub(r'[^\w]', ' ', string).split())
  temp = vals.split()
  return temp[-1]

d = collections.defaultdict(list)
columns = ['Название продукта', 'Название категории продукта', 'Цену продукта', 'Название города']
nazvanie = []
category_product = []
price_product = []
city = []
k = 0

for urls in all_links:
  soup_2 = BeautifulSoup(requests.get(urls).content, 'html.parser')
  print(urls)
  nazvanie.append(soup_2.find_all('h1', {'class':'title'})[0].text)
  category = soup_2.find_all('li', itemprop='itemListElement')[2].find_all('a')[0].text
  category_product.append(category.replace('\n',""))
  price_product.append(soup_2.find_all('div',{'class':'new'})[0].text)
  value = ''
  try:
    value = country(soup_2.find_all('div',{'class':'goodsDescription_short'})[0].text)
    if value[0].isupper() == False:
      city.append("")
    else:
      city.append(value)
  except IndexError:
    city.append('')
  


https://kingfisher.kz/moreprodukty/krevetki/krevetka-korolevskaya-2630-ochischennaya/
https://kingfisher.kz/moreprodukty/krevetki/krevetka-korolevskaya-ochischennaya-31-40/
https://kingfisher.kz/moreprodukty/krevetki/langustiny-v-pancire-s-golovoy-razmery-21-30/
https://kingfisher.kz/moreprodukty/krevetki/krevetka-korolevskaya-2125-v-pancire-bez-golovy/
https://kingfisher.kz/bakaleya/aziatskaya-kuhnya/vodorosli/chuka-salat-zamorozhenniy/
https://kingfisher.kz/moreprodukty/krevetki/krevetka-arkticheskaya-7090-grenlandiya/
https://kingfisher.kz/moreprodukty/krevetki/krevetka-tigrovaya-1620-v-pancire/
https://kingfisher.kz/moreprodukty/krevetki/langustiny-v-pancire-s-golovoy-razmer-10-20/
https://kingfisher.kz/moreprodukty/krevetki/krevetka-salatnaya-100-200/
https://kingfisher.kz/moreprodukty/krevetki/krevetka-severnaya-90-120-grenlandiya/
https://kingfisher.kz/moreprodukty/krevetki/langustiny-v-pancire-bez-golovy-razmer-30-55/
https://kingfisher.kz/moreprodukty/krevetki/krevetka-tigrova

""


In [ ]:
d[columns[0]] = nazvanie
d[columns[1]] = category_product
d[columns[2]] = price_product
d[columns[3]] = city

In [163]:
data = pd.DataFrame(d)
data.head()

,Название продукта,Название категории продукта,Цену продукта,Название города
0,Креветки Королевские 26/30 очищенные,Креветки,5 290 тг.,Индия
1,Креветка Королевская 31/40 очищенная,Креветки,5 900 тг.,Индия
2,"Лангустины в панцире с головой, размер 21-30",Креветки,6 500 тг.,
3,"Креветка Королевская 21/25, в панцире без головы",Креветки,5 500 тг.,Индия
4,Чука салат замороженный,Азиатская Кухня,2 800 тг.,
